In [1]:
from trajectree.fock_optics.utils import create_vacuum_state
from trajectree.fock_optics.devices import ry, rx, rz, global_phase, generalized_mode_mixer_unitary
from trajectree.fock_optics.measurement import bell_state_measurement
from trajectree.fock_optics.outputs import print_quantum_state, read_quantum_state
from trajectree.fock_optics.noise_models import single_mode_bosonic_noise_channels, depolarizing_operators, two_qubit_depolarizing_channel
from trajectree.fock_optics.light_sources import light_source

from trajectree.trajectory import trajectory_evaluator, quantum_channel

import numpy as np
from scipy import sparse as sp
import qutip as qt
from numpy import sqrt
from quimb.tensor.tensor_arbgeom import tensor_network_apply_op_vec #type: ignore
from quimb.tensor import MatrixProductOperator as mpo #type: ignore

import matplotlib.pyplot as plt
import matplotlib.animation as animation

import copy
from IPython.display import HTML
from IPython.display import Image

In [2]:
N = 2
num_modes = 4
efficiency = 0.5
error_tolerance = 1e-10
num_simulations = 1

bsm_det_effs = [0.5,0.5]
bsm_dark_counts_gain = [1+1e-6,1+1e-6]
# bsm_measurements = {0:(2,3), 1:(0,1)}
bsm_measurements = {0:(0,3), 1:(1,2)}



In [3]:
mean_photon_num = 0.1
vacuum = create_vacuum_state(num_modes=8, N=N)
psi, _ = light_source(vacuum, N, mean_photon_num, 8, error_tolerance, compress=True, contract=True)

read_quantum_state(psi, N, num_states = 2)

Corresponding Basis terms:
0H0V_B 0H0V_A - 0 - [0.90625174+0.j]
1H0V_B 1H0V_A - 5 - [-0.29147816+0.j]
0H1V_B 0H1V_A - 10 - [-0.29147816+0.j]
1H1V_B 1H1V_A - 15 - [0.09374826+0.j]


In [ ]:
quantum_channel_list = []
damping_kraus_ops_0 = single_mode_bosonic_noise_channels(noise_parameter = 1-efficiency, N = N)
damping_kraus_ops_1 = single_mode_bosonic_noise_channels(noise_parameter = 1-efficiency, N = N)
two_mode_kraus_ops_0 = [sp.kron(op1, op2) for op1 in damping_kraus_ops_0 for op2 in damping_kraus_ops_0]
two_mode_kraus_ops_1 = [sp.kron(op1, op2) for op1 in damping_kraus_ops_1 for op2 in damping_kraus_ops_1]
quantum_channel_list.append(quantum_channel(N = N, num_modes = num_modes, formalism = "kraus", kraus_ops_tuple = ((0,1), two_mode_kraus_ops_0), name = "DIneff - 0")) # The tuples in this list are defined as (sites, kraus_ops). The sites are the sites where the Kraus ops are applied.
quantum_channel_list.append(quantum_channel(N = N, num_modes = num_modes, formalism = "kraus", kraus_ops_tuple = ((2,3), two_mode_kraus_ops_1), name = "detector inefficiency - 1")) # The tuples in this list are defined as (sites, kraus_ops). The sites are the sites where the Kraus ops are applied.

evaluator = trajectory_evaluator(quantum_channel_list)


Corresponding Basis terms:
0H0V_B 0H0V_A - 0 - [0.90625174+0.j]
1H0V_B 1H0V_A - 5 - [-0.29147816+0.j]
0H1V_B 0H1V_A - 10 - [-0.29147816+0.j]
1H1V_B 1H1V_A - 15 - [0.09374826+0.j]
Corresponding Basis terms:
0H0V_B 0H1V_A - 2 - [-0.98731475+0.j]
1H0V_B 1H1V_A - 7 - [0.15877525-0.j]


In [184]:
read_quantum_state(psi, N, num_states = 2)

psi_iter = evaluator.perform_simulation(psi, error_tolerance, normalize = True)
evaluator.graph.show()

print(evaluator.traversed_nodes)

read_quantum_state(psi_iter, N, num_states = 2)

Corresponding Basis terms:
0H0V_B 0H0V_A - 0 - [0.90625174+0.j]
1H0V_B 1H0V_A - 5 - [-0.29147816+0.j]
0H1V_B 0H1V_A - 10 - [-0.29147816+0.j]
1H1V_B 1H1V_A - 15 - [0.09374826+0.j]
()
├── (0,)
├── (1,)
└── (2,)

(0, 0)
Corresponding Basis terms:
0H0V_B 0H0V_A - 0 - [0.97479042+0.j]
1H0V_B 1H0V_A - 5 - [-0.15676114+0.j]
0H1V_B 0H1V_A - 10 - [-0.15676114+0.j]
1H1V_B 1H1V_A - 15 - [0.02520958+0.j]


In [17]:
print(damping_kraus_ops_0[1])

read_quantum_state(psi, N, num_states = 2)
psi_dense = read_quantum_state(psi, N, num_states = 2, return_dense = True)

<Compressed Sparse Row sparse array of dtype 'complex128'
	with 1 stored elements and shape (2, 2)>
  Coords	Values
  (0, 1)	(0.7071067811865476+0j)
Corresponding Basis terms:
0H0V_B 0H0V_A - 0 - [0.90625174+0.j]
1H0V_B 1H0V_A - 5 - [-0.29147816+0.j]
0H1V_B 0H1V_A - 10 - [-0.29147816+0.j]
1H1V_B 1H1V_A - 15 - [0.09374826+0.j]


All that we saw from the above experiment is that the amplitude damping operator generates states with lower photon numbers. We compare the actual state against direct matrix multiplication operations in the next cell. Nothing else is tested here per se. 


In [186]:
# print(two_mode_kraus_ops_0[0].toarray())
# print(psi_dense.shape)

new_dense_state = np.kron(two_mode_kraus_ops_0[0].toarray(), two_mode_kraus_ops_0[0].toarray()) @ psi_dense

print(sp.csr_array(np.kron(two_mode_kraus_ops_0[0].toarray(), two_mode_kraus_ops_0[0].toarray())))

dense_state = np.reshape(new_dense_state.data, (-1, 1), order = 'C')
dense_state = sp.csr_matrix(dense_state)
dense_state.data = np.round(dense_state.data, 10)
dense_state.eliminate_zeros()
print_quantum_state(N, dense_state, 2, False)

<Compressed Sparse Row sparse array of dtype 'complex128'
	with 16 stored elements and shape (16, 16)>
  Coords	Values
  (0, 0)	(1+0j)
  (1, 1)	(0.7071067811865476+0j)
  (2, 2)	(0.7071067811865476+0j)
  (3, 3)	(0.5000000000000001+0j)
  (4, 4)	(0.7071067811865476+0j)
  (5, 5)	(0.5000000000000001+0j)
  (6, 6)	(0.5000000000000001+0j)
  (7, 7)	(0.35355339059327384+0j)
  (8, 8)	(0.7071067811865476+0j)
  (9, 9)	(0.5000000000000001+0j)
  (10, 10)	(0.5000000000000001+0j)
  (11, 11)	(0.35355339059327384+0j)
  (12, 12)	(0.5000000000000001+0j)
  (13, 13)	(0.35355339059327384+0j)
  (14, 14)	(0.35355339059327384+0j)
  (15, 15)	(0.2500000000000001+0j)
Corresponding Basis terms:
0H0V_B 0H0V_A - 0 - [0.90625174+0.j]
1H0V_B 1H0V_A - 5 - [-0.14573908+0.j]
0H1V_B 0H1V_A - 10 - [-0.14573908+0.j]
1H1V_B 1H1V_A - 15 - [0.02343706+0.j]
